* Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All). Or, alternatively, **Restart & Run All**.

* Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

* You can always add additional cells to the notebook to experiment, to test your answers, or to provide additional support for your answers.

* You should not need to install new packages to complete an assignment. If you use any packages not available via the provided `Project.toml`, or if you change the `Manifest.toml`, then your assignment will likely not be graded correctly.

* Submissions are only accepted via CANVAS!

* Late submissions will not be accepted and receive 0%. There will be no exceptions.

* By entering your name below you confirm that you have completed this assignment on your own. Suspicion of plagiarism / copying may result in an ad hoc oral examination on the content of the assignment. I may then downgrade an assignment. 

In [ ]:
STUDENTID = ""

---

# MATH 405/607 

# Numerical Methods for Differential Equations

## Assignment 4: Finite difference methods for BVPs and IBVPs


* This assignment will be worth 10% of the overall grade.
* You must submit both an `.ipynb` notebook **AND** a PDF generated from that notebook that contains all output.

In [ ]:
include("math405.jl")

### Question 1: A 'singularly perturbed' advection-diffusion problem [5+5+5]

Consider the boundary value problem
$$\begin{aligned}
   u_x - \epsilon u_{xx} &= 1, \qquad x \in (0, 1) \\ 
   u(0) &= u(1) = 0.
\end{aligned}$$

You may use, without proof, that the unique solution is given by 
$$
    u(x) =  x - \frac{e^{x/\epsilon} - 1}{e^{1/\epsilon} - 1}
$$

(a) Discretise the domain with $x_n = n h$, $h = 1/N$, $n = 0, \dots, N$, then formulate a finite-difference discretisation of the BVP, using a centered finite-difference approximation for $u_x$ (cf. A1)

(b) Define the *truncation error* $T_n$ for your discretisation. 

<!-- (c) Estimate the trunction error $T_n$ in terms of $h$ and derivatives of $u$. Note: You can reuse results from the lectures and from A1.
 -->
If defined correctly, the truncation error $T_n$ will satisfy 
$$
|T_n| \leq C h^2 \big( \| u_{xxx} \|_\infty + \epsilon  \| u_{xxxx} \|_{\infty} \big) 
$$
You need not prove this. (However it would be a good exercise to do so.)
 
(c) Estimate the truncation error $T_n$ in terms of $h$ and $\epsilon$. Suppose we want to achieve some tolerance $c \tau$ with some unknown constant $c$. Given $\epsilon$, how should we choose $h$? I.e., how should we balance $h$ and $\epsilon$?

(Hint: from the exact solution above we can bound $\| u^{(j)} \|_\infty \leq 2\epsilon^{-j}$ for $j \geq 2$, where $u^{(j)}$ denotes the $j$-th derivative of $u$) 

YOUR ANSWER HERE

### Question 1 continued [5] 

(d) Implement the method derived in Question 1 (a) as a function taking the parameters $N, \epsilon$ as input and return the finite difference solution $U$ as vector of the *interior nodes only*, i.e., $U = (U_1, \dots, U_{N-1})$, where $U_n \approx u(x_n)$. 

In [ ]:
function steady_advection_diffusion(N, epsn)
    h =  1/N
    # YOUR CODE HERE
    return U
end


### Question 1 continued [10] 

(e) Produce a figure that provides a graphical confirmation of the joint convergence rate in $h$ **and** $\epsilon$. To get you started, there is some code provided that collects data in the form of $(h, \epsilon, {\rm error})$, and which you now need to visualize. You can use this as is, or modify the code to collect different data. There are multiple ways to solve this question. 

(To understand the provided code you will need to solve Question 1c in detail.)

In [ ]:
"""Computes finite difference solution with N+1 grid points 
and returns the error in the max norm."""
function get_err(N, epsn)
    u = x -> x - (exp(x/epsn) - 1) /  (exp(1/epsn) - 1)
    X = range(0, 1, length=N+1)[2:end-1]
    U = steady_advection_diffusion(N, epsn)
    return norm(U - u.(X), Inf)
end 

H = []
Epsn = [] 
Err = []
for epsn in [1.0, 0.3, 0.1, 0.03, 0.01]
    for tau in [10.0, 3.0, 1.0, 0.3, 0.1, 0.03, 0.01, 0.003, 0.001]
        h = tau * epsn^(-2/3)
        N = max(10, ceil(Int, 1/h))
        push!(H, 1/N)
        push!(Epsn, epsn)
        push!(Err, get_err(N, epsn))
    end
end

# Now use the error data collected in the above loop to 
# produce a convergence graph. The task is to come up with a 
# figure that clearly visually communicates the joint 
# asymptotics in h and ϵ in the error estimate from (c)
# E.g. you could produce a scatter plot overlayed by the 
# theoretical prediction.

# YOUR CODE HERE

----

### Question 2: Advection with Boundary Condition  [5+7+8]

For $a > 0$ consider the advection equation with an inflow boundary at $x = 0$ and an outflow boundary at $x = 1$:

$$
\begin{aligned}
    u_t + a u_x &= 0,  \qquad x \in (0, 1), t \in (0, 1), \\ 
    u(0, x) &= u_0(x), \\ 
    u(t, 0) &= q(t).
\end{aligned}
$$

(a) Adapt the upwind discretisation from the lecture for this equation and write down the full discrete system.

(b) Define the truncation error $T_n^m$ and prove that 
$$
    |T_n^m| \leq C\big( c_h h + c_\tau \tau \big)
$$
specifying the constants $c_h, c_\tau$ in terms of the max-norm of derivatives of $u$.

(c) Follow the stability and converge proof in the lectures to deduce that 
$$
    \max_{n, m} |U_n^m - u(m\tau, nh)| \leq C(h + \tau),
$$
To prove this you will need to use the CFL condition. Clearly state it and point out where you are using it.

YOUR ANSWER HERE

### Question 2 continued [10] 

(d) Convergence study: 
Let $a = 1.3$ and consider an exact solution $u(t, x) = \sin(4*(x - a t))$ to the homogeneous solution. 
Determine the corresponding initial and boundary values $u_0(x)$ and $q(t)$. 
Then implement a function that takes `h` or `N` as input, then determines a suitable time-step $\tau$ and then solves the finite-difference scheme from part (a) for $0 \leq t \leq 1$. Compute the error, plot it against $N$, and confirm that the rate you obtain is the rate that you predicted. 

As general guidance, your code could like something like this: 
```julia
const a = 1.3 
u0(x) = 
q(t) = 
u(t, x) = 

function compute_error(N)
    X = range(0, 1, length=N+1)
    h = 1/N
    τ =         # choose a good time-step, not too large, not too small    
    M = ceil(Int, 1/τ)    # it is ok to solve up to a time slightly > 1
    U = u0.(X)     # the initial condition 
    err = 0.0 
    for m = 1:M
        U[2:end] = # ... the finite-difference scheme
        U[1] = # the boundary condition 
        err = max(err, # measure the error at the current time 
    end
    return err 
end

NN =   # choose a suitable collection of N values
err = compute_error.(NN)
plot(NN, err, label = "error", 
    lw=2, m=:o,ms=6, yscale=:log10, xscale=:log10, size = (500, 350))
plot!(NN[3:5], 3*NN[3:5].^(-1), label = "predicted rate", 
      lw=2, c=:black, ls=:dash)
```

In [ ]:
# YOUR CODE HERE

---

### Question  3: Crank-Nicholson Method [5+5+10]

Consider the model diffusion equation 
$$\begin{aligned}
    u_t(t, x) &= u_{xx}(t, x) + f(t, x), \\ 
    u(0, x) &= u_0(x), \\ 
    u(t, 0) &= u(t, 1) = 0
\end{aligned}$$
for $x \in (0, 1)$ and $t \in (0, 1)$, $u_0 : [0, 1] \to \mathbb{R}$, $f : [0, 1] \times [0, 1] \to \mathbb{R}$.

(a) Discretise it using second-order centered finite-differences in space and the Crank-Nicholson scheme (cf L09, L10) in time. Write down the full set of equations for the *interior* degrees of freedom, $U_n^m \approx u(m \tau, n h)$, $n = 1, \dots, N$, where $h$ is the spatial grid spacing and $\tau$ the time-step.

(b) Define the truncation error $T_n^m$. 

(c) It can be shown that the truncation error satisfies
$$
    |T_n^m| \leq C  \big( h^2 +\tau^2 \big).
$$
This is a long and tedious proof which you don't need to carry out. Instead, use the max-norm stability technique (*NOT* the von Neumann stability!!) from L13 to prove that 
$$
    \| U - u \|_\infty := \max_{n, m} |U_n^m - u(\tau m, h n)|
$$
 $n = 0, \dots, N$ and $m = 0, \dots, M := \lceil 1/\tau \rceil$. You will need a restriction on the time-step $\tau$ in terms of $h$. Make sure you state precisely what that restriction is. 
 
*Note: we will continue the discussion of this restriction after some von Neumann analysis in Q5.*

YOUR ANSWER HERE

---------


### Question 4: Review Fourier Analysis [5+5+5]

We consider sequences $(U_n)_{n \in \mathbb{Z}}$ where we think of $U_n \approx u(h n)$. The semi-discrete Fourier transform is given by 
$$
    \hat{U}(k) := h \sum_{n \in \mathbb{Z}} U_n e^{i k h n}, \qquad k \in (-\pi/h, \pi/h].
$$
We can think of the mapping $U \mapsto \hat{U}$ as a coordinate transformation or a basis transformation. A (homogeneous) finite-difference scheme in terms of $U$ can be rewritten in terms of $\hat{U}$ and becomes *diagonal*. In preparation, we now transform three finite-difference operators into reciprocal space:

**Note:** The semi-discrete Fourier transform is more commonly defined as $\hat{U}(k) := \sum_{n \in \mathbb{Z}} U_n e^{-i k h n}.$ Since the choice of sign is a bit arbitrary and I used the $e^{i k h n}$ definition in the lectures, I am sticking with it for the assignment as well. But please be aware that most references you look at will use the $e^{- i k h n}$ definition.

(a) Let $V_n := U_n - U_{n-1}$; prove that 
$$
    \hat{V}(k) = (1 - e^{ikh}) \hat{U}(k)
$$

(b) Let $V_n := U_{n+1} - U_{n-1}$; prove that 
$$
    \hat{V}(k) = 2i \sin(kh) \hat{U}(k)
$$

(c) Let $V_n := U_{n+1} - 2 U_n + U_{n-1}$; prove that 
$$
    \hat{V}(k) = - 4 \sin^2(kh/2) \hat{U}(k).
$$


YOUR ANSWER HERE

<a name = 'q7'></a>

### Question 5: Von Neuman Stability Analysis [8+7] 

All finite-difference discretisations of various advection-diffusion systems we have considered so far can be written in the form 
$$
    [A U^{m+1}]_n = [B U^m]_n,
$$
where $A, B$ are finite-difference operators. By writing $\widehat{A U}(k) =
\hat{a}(k) \hat{U}(k)$ we obtain 
$$
    \hat{a}(k) \hat{U}^{m+1}(k) = \hat{b}(k) \hat{U}^m(k), 
$$
Thus, we obtain the stability estimate 
$$
    \| \hat{U}^{m+1} \|_{2} \leq \| \hat{b}/\hat{a} \|_\infty \| \hat{U}^m \|_{2},
$$
and using the Plancherel theorem, 
$$
    \| U^{m+1} \|_{2} \leq \| \hat{b}/\hat{a} \|_\infty \| U^m \|_{2}.
$$
A method is called *von Neumann stable* if $\| \hat{b}/\hat{a} \|_\infty \leq 1$. It is unconditionally stable, if there are no restrictions on the step size. In the following we look at two concrete examples. 

HINT: Use the results of Question 4!

(a) Crank-Nicholson Scheme: show that the Crank-Nicholson method from Question 2, but now applied to the infinite grid $h\mathbb{Z}$, is unconditionally von Neumann stable.

(b) Consider the centered finite difference approximation of the advection equation $u_t + a u_x = 0$, 
$$
    \frac{U^{m+1}_n - U^m_n}{\tau} + a \frac{U^m_{n+1} - U^m_{n-1}}{2h} = 0, 
    \qquad n \in \mathbb{Z}, m = 0, 1, \dots, M-1,
$$
Prove that, given $h$, there is no choice of $\tau$ for which this scheme is von Neumann stable. I.e. the scheme is unconditionally unstable.


YOUR ANSWER HERE

---

### Question 3 continued: vN Stability [0]

This final part of the assignment will not be graded and is only intended for those who are interested getting a bit deeper into the subject. 

Imagine you are applying the CN scheme to the diffusion equation on all of $\mathbb{R}$ resulting in the discretisation on the infinite grid $h \mathbb{Z}$. Under some technical assumptions and with (a fair bit of) extra work you can prove a consistency error estimate in the 2-norm, of the form, 
$$
  h^{1/2} \| T^m \|_2 \leq C(h^2 + \tau^2)
$$
This is reasonable to expect from the max-norm estimate but non-trivial because of issues related to the infinite domain. 

Use this, combined with the vN stability analysis from Q5 to prove that --- without any conditions on $h, \tau$ --- the CN scheme satissfies 
$$
    \max_{m=0,\dots,M}  \bigg( h\sum_{n \in \mathbb{Z}} |u(t_m, x_n) - U_{m,n}|^2 \bigg)^{1/2} \leq C (\tau M) (h^2 + \tau^2).
$$